## KITTI: Visualizer Raw Data or Pre-Processed Data

In [ ]:
import sys
sys.path.insert(0, '/home/rlab10/OpenPCDet')

from pcdet.datasets.kitti.kitti_dataset_custom import *
from pcdet.datasets.dataset import *
from pcdet.datasets import DatasetTemplate
from pcdet.utils import common_utils, calibration_kitti, box_utils

import yaml
from easydict import EasyDict
from pathlib import Path
import pickle
from IPython import get_ipython

# Somehow without this my Open3D visualization didn't started.
import os
# CPU
# os.environ["LIBGL_ALWAYS_INDIRECT"]="0"
# os.environ["MESA_GL_VERSION_OVERRIDE"]="4.5"
# os.environ["MESA_GLSL_VERSION_OVERRIDE"]="450"
# os.environ["LIBGL_ALWAYS_SOFTWARE"]="1"

# GPU
# source: https://github.com/isl-org/Open3D/issues/5126#issuecomment-3374337980, https://github.com/PrismLauncher/PrismLauncher/issues/866#issuecomment-1432251741
# https://github.com/isl-org/Open3D/issues/6872
os.environ["XDG_SESSION_TYPE"]="x11"
os.environ["GDK_BACKEND"]="x11"
os.environ["MESA_D3D12_DEFAULT_ADAPTER_NAME"]="NVIDIA"

from tools.visual_utils import open3d_vis_utils as Visualizer

dataset_cfg=EasyDict(yaml.safe_load(open('/home/rlab10/OpenPCDet/tools/cfgs/dataset_configs/kitti_dataset_custom.yaml')))
class_names=['Car', 'Pedestrian', 'Cyclist']
file_path = '/home/rlab10/OpenPCDet/pcdet/datasets/kitti/kitti_dataset_custom.py' 
# /home/rlab10/OpenPCDet
ROOT_DIR = (Path(file_path).resolve().parent / '../../../').resolve()
data_path = ROOT_DIR / 'data' / 'kitti'
ext = '.bin'
data = []
num_features = len(dataset_cfg.POINT_FEATURE_ENCODING.src_feature_list)

class DemoDatasetKitti(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=True, root_path=None, logger=None, data_processor_flag=False, fov_mode=False):
        """
        Args:
            root_path:
            dataset_cfg:
            class_names:
            training:
            logger:
        """
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )
        self.split = self.dataset_cfg.DATA_SPLIT['train']
        self.root_split_path = self.root_path / ('training' if self.split != 'test' else 'testing')
        split_dir = self.root_path / 'ImageSets' / (self.split + '.txt')
        self.sample_id_list = [x.strip() for x in open(split_dir).readlines()] if split_dir.exists() else None
        self.demo_kitti_infos = []
        self.include_kitti_data(mode='train')
        self.data_processor_flag = data_processor_flag
        self.fov_mode = fov_mode

    def include_kitti_data(self, mode):
        if self.logger is not None:
            self.logger.info('DemoDatasetKitti: Loading raw KITTI dataset')
        demo_kitti_infos = []

        for info_path in self.dataset_cfg.INFO_PATH[mode]: # 'train', bc training=True
            info_path = self.root_path / info_path
            if not info_path.exists():
                continue
            # Read the data infos from kitti_infos_train.pkl
            with open(info_path, 'rb') as f:
                infos = pickle.load(f)
                demo_kitti_infos.extend(infos)
        # Add the newly loaded KITTI dataset information to kitti_infos list.
        self.demo_kitti_infos.extend(demo_kitti_infos)

    def __len__(self):
        return len(self.sample_id_list)
    
    def get_lidar(self, idx):
        lidar_file = self.root_split_path / 'velodyne' / ('%s.bin' % idx)
        assert lidar_file.exists()
        return np.fromfile(str(lidar_file), dtype=np.float32).reshape(-1, 4)

    def get_calib(self, idx):
        calib_file = self.root_split_path / 'calib' / ('%s.txt' % idx)
        assert calib_file.exists()
        return calibration_kitti.Calibration(calib_file)

    @staticmethod
    def get_fov_flag(pts_rect, img_shape, calib):
        """
        Args:
            pts_rect:
            img_shape:
            calib:

        Returns:

        """
        pts_img, pts_rect_depth = calib.rect_to_img(pts_rect)
        val_flag_1 = np.logical_and(pts_img[:, 0] >= 0, pts_img[:, 0] < img_shape[1])
        val_flag_2 = np.logical_and(pts_img[:, 1] >= 0, pts_img[:, 1] < img_shape[0])
        val_flag_merge = np.logical_and(val_flag_1, val_flag_2)
        pts_valid_flag = np.logical_and(val_flag_merge, pts_rect_depth >= 0)

        return pts_valid_flag

    def prepare_demo_data(self, data_dict):
        print('DemoDatasetKitti: prepare_demo_data() called')
        if 'gt_boxes' not in data_dict:
            assert 'gt_boxes' in data_dict, 'gt_boxes should be provided for demo visualization'
        else:         
            if data_dict.get('gt_boxes', None) is not None:
                selected = common_utils.keep_arrays_by_name(data_dict['gt_names'], self.class_names)
                data_dict['gt_boxes'] = data_dict['gt_boxes'][selected]
                data_dict['gt_names'] = data_dict['gt_names'][selected]
                gt_classes = np.array([self.class_names.index(n) + 1 for n in data_dict['gt_names']], dtype=np.int32)
                # already transformed 3D boxes LiDAR coord. + add number for the class
                gt_boxes = np.concatenate((data_dict['gt_boxes'], gt_classes.reshape(-1, 1).astype(np.float32)), axis=1)
                data_dict['gt_boxes'] = gt_boxes


            if data_dict.get('points', None) is not None:
                data_dict = self.point_feature_encoder.forward(data_dict)
            
            if self.data_processor_flag:
                print('DemoDatasetKitti: data processing activated')
                data_dict = self.data_processor.forward(data_dict=data_dict)
    
        return data_dict

    def __getitem__(self, index):
        print('DemoDatasetKitti: __getitem__ called')
        if self.merge_all_iters_to_one_epoch:
            index = index % len(self.demo_kitti_infos)
        # works
        info = copy.deepcopy(self.demo_kitti_infos[index])

        sample_idx = info['point_cloud']['lidar_idx']
        img_shape = info['image']['image_shape']
        calib = self.get_calib(sample_idx)
        get_item_list = self.dataset_cfg.get('GET_ITEM_LIST', ['points'])

        input_dict = {
            'frame_id': sample_idx,
        }

        if 'annos' in info:
            annos = info['annos']
            annos = common_utils.drop_info_with_name(annos, name = 'DontCare')
            loc, dims, rots = annos['location'], annos['dimensions'], annos['rotation_y']
            gt_names = annos['name']
            gt_boxes_camera = np.concatenate([loc, dims, rots[...,np.newaxis]], axis = 1).astype(np.float32)
            gt_boxes_lidar = box_utils.boxes3d_kitti_camera_to_lidar(gt_boxes_camera, calib)

        input_dict.update({
            'gt_names': gt_names,
            'gt_boxes': gt_boxes_lidar
        })

        if 'points' in get_item_list:
            points = self.get_lidar(sample_idx)

            if self.fov_mode:
                print('DemoDatasetKitti: fov mode activated')
                # only the points in the FOV of the camera are important for me
                pts_rect = calib.lidar_to_rect(points[:, 0:3])
                fov_flag = self.get_fov_flag(pts_rect, img_shape, calib)
                points = points[fov_flag]
            input_dict['points'] = points

        data_dict = self.prepare_demo_data(data_dict=input_dict)

        return data_dict

def main(mode=str):
    logger = common_utils.create_logger()
    if mode == 'raw':
        logger.info('-----------------Quick Visualizer Demo of raw data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        demo_dataset = DemoDatasetKitti(dataset_cfg=dataset_cfg, class_names=class_names, training=False, root_path=data_path, logger=logger, data_processor_flag=False, fov_mode=False)
        logger.info(f'Total number of samples: \t{len(demo_dataset)}')
        # raw data not  0-4
        #demo_dataset.demo_kitti_infos[4]
        #demo_dataset[0]
        # get_infos oder prepare_data hier implementieren(flow: include_kitti_data->__getitem__->prepare_data)
        # or using KittiDataset for accessing the
        data_dict = demo_dataset[4]
        print('frame_id: ', data_dict['frame_id'])
        points = data_dict['points']
        gt_boxes = data_dict['gt_boxes']
        gt_names = data_dict['gt_names']

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes, gt_labels=gt_names, point_colors=None, draw_origin=True)
        logger.info('Demo visualization of raw data done.')

    elif mode == 'pre-processed':
        logger.info('-----------------Quick Visualizer Demo of pre-processed data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        demo_dataset = DemoDatasetKitti(dataset_cfg=dataset_cfg, class_names=class_names, training=True, root_path=data_path, logger=logger, data_processor_flag=True, fov_mode=False)
        logger.info(f'Total number of samples: \t{len(demo_dataset)}')
       
        data_dict = demo_dataset[4]
        print('frame_id: ', data_dict['frame_id'])
        points = data_dict['points']
        gt_boxes = data_dict['gt_boxes']
        gt_names = data_dict['gt_names']

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes, gt_labels=gt_names, point_colors=None, draw_origin=True)
        logger.info('Demo visualization of pre-processed data done.')

    elif mode == 'val':
        logger.info('-----------------Quick Visualizer Demo of validation .pkl data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        #pkl_path = ROOT_DIR / 'data' / 'kitti' / 'data_test_pipeline' / 'visualizer' / 'kitti_val_dataset.pkl'
        pkl_path = ROOT_DIR / 'data' / 'kitti' / 'kitti_val_dataset.pkl'
        #pkl_path = ROOT_DIR / 'data' / 'kitti' / 'Domain Generalization' / 'densification' / 'kitti_val_dataset_3x_densified.pkl'
        #pkl_path = ROOT_DIR / 'data' / 'kitti' / 'Domain Generalization' / 'random beam re-sampling' / 'kitti_val_dataset_rbrs.pkl'
        #pkl_path = ROOT_DIR / 'data' / 'kitti' / 'Domain Generalization' / 'pdrw interpolation' / 'kitti_val_dataset_pdrw.pkl'
        
        with open(pkl_path, 'rb') as f:
            demo_val_dataset = pickle.load(f)

        data_dict = demo_val_dataset[4]
        points = data_dict['points']
        gt_boxes_lidar = data_dict['annos']['gt_boxes_lidar']
        gt_names = data_dict['annos']['name']
        gt_scores = data_dict['annos']['score']
        lidar_idx = data_dict['point_cloud']['lidar_idx']
        print(lidar_idx)

        #class_n = data_dict['annos']['name']
        #print(f"\nLidar IDX: {lidar_idx}")
        #print(f"Points: {points}\n")
        #print(f"Class Names: {class_n}\n")
        #print(f"Ground Truth Boxes (LiDAR): {gt_boxes_lidar}")

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes_lidar, gt_labels=gt_names, gt_score=gt_scores, point_colors=None, draw_origin=True, vc='val')
        logger.info('Demo visualization of pre-processed validation data done.')

    elif mode == 'train':
        logger.info('-----------------Quick Visualizer Demo of training .pkl data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        #pkl_path = ROOT_DIR / 'data' / 'kitti' / 'data_test_pipeline' / 'visualizer' / 'kitti_train_dataset.pkl'
        pkl_path = ROOT_DIR / 'data' / 'kitti' / 'kitti_train_dataset.pkl'
        #pkl_path = ROOT_DIR / 'data' / 'kitti' / 'Domain Generalization' / 'densification' / 'kitti_train_dataset_3x_densified.pkl'
        #pkl_path = ROOT_DIR / 'data' / 'kitti' / 'Domain Generalization' / 'random beam re-sampling' / 'kitti_train_dataset_rbrs.pkl'
        #pkl_path = ROOT_DIR / 'data' / 'kitti' / 'Domain Generalization' / 'pdrw interpolation' / 'kitti_train_dataset_pdrw.pkl' # points_interp
    
        with open(pkl_path, 'rb') as f:
            demo_train_dataset = pickle.load(f)

        data_list = demo_train_dataset[4] # specific sample from row no. in train.txt, e.g. (34-1) - 000067)
        # [0] - original; [1] - gt_sampling; [2] - flip_x; [3] - w_rotation; [4] - l_rotation; 
        # [5] - l_scaling; [6] - w_translation
        try:
            print("\nPlease select an option for the visualization:")
            print('[0] Original')
            print('[1] Ground Truth Sampling (gt_sampling)')
            print('[2] Flip along X-axis (flip_x)')
            print('[3] World rotation (w_rotation)')
            print('[4] Local rotation (l_rotation)')
            print('[5] Local scaling (l_scaling)')
            print('[6] World translation (w_translation)\n')
            user_choice = int(input('Your selection: '))
            if user_choice not in range(7):
                raise ValueError('Invalid selection')
        except ValueError as e:
            print(f"Error: {e}. Please enter a number between 0 and 6.")
            logger.error("Invalid user input for the visualization mode.")
            return
        
        switch_cases = {0: "original", 1: "gt_sampling", 2: "flip_x", 3: "w_rotation", 4: "l_rotation", 5: "l_scaling", 6: "w_translation"}
        
        selected_data = data_list[user_choice]
        points = selected_data['points'] 
        gt_boxes = selected_data['gt_boxes']
        gt_names = selected_data['gt_names']
        gt_scores = selected_data['cam_info']['score'] # take it from annotations (Camera frame)
        frame_id = selected_data['frame_id']
        print(frame_id)

        if user_choice in [3, 4]: 
            noise_key = 'noise_world_rotation' if user_choice == 3 else 'noise_local_rotation'
            noise_rot = selected_data.get(noise_key, None)
            if noise_rot is not None:
                logger.info(f'Noise rotation for {switch_cases[user_choice]}: {noise_rot}')
        if user_choice == 5:
            noise_scale = selected_data.get('noise_local_scaling', None)
            if noise_scale is not None:
                logger.info(f'Noise scaling for {switch_cases[user_choice]}: {noise_scale}')
        if user_choice == 6:
            noise_trans = selected_data.get('noise_world_translation', None)
            if noise_trans is not None:
                logger.info(f'Noise translation for {switch_cases[user_choice]}: {noise_trans}')
        

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes, gt_labels=gt_names, gt_score=gt_scores, point_colors=None, draw_origin=True, vc='')
        logger.info('Demo visualization of training data done.')
    
    if get_ipython():
        get_ipython().run_line_magic('reset', '-sf')
        #get_ipython().kernel.do_shutdown(restart=False)

if __name__ == '__main__':
    #main(mode='raw')
    #main(mode='pre-processed')
    #main(mode='val')
    main(mode='train')

## ZOD: Visualizer Raw Data or Pre-Processed Data

In [1]:
import sys
sys.path.insert(0, '/home/rlab10/OpenPCDet')

from pcdet.datasets.zod.zod_dataset_custom import *
from pcdet.datasets.dataset import *
from pcdet.datasets import DatasetTemplate
from pcdet.utils import common_utils

from zod import ZodFrames
from zod.constants import Camera, Lidar
from zod.utils.geometry import get_points_in_camera_fov, transform_points
from zod.data_classes.geometry import Pose

import yaml
from easydict import EasyDict
from pathlib import Path
import pickle
from IPython import get_ipython

# Somehow without this my Open3D visualization didn't started.
import os
# CPU
#os.environ["LIBGL_ALWAYS_INDIRECT"]="0"
#os.environ["MESA_GL_VERSION_OVERRIDE"]="4.5"
#os.environ["MESA_GLSL_VERSION_OVERRIDE"]="450"
#os.environ["LIBGL_ALWAYS_SOFTWARE"]="1"

# GPU
# source: https://github.com/isl-org/Open3D/issues/5126#issuecomment-3374337980, https://github.com/PrismLauncher/PrismLauncher/issues/866#issuecomment-1432251741
# https://github.com/isl-org/Open3D/issues/6872
os.environ["XDG_SESSION_TYPE"]="x11"
os.environ["GDK_BACKEND"]="x11"
os.environ["MESA_D3D12_DEFAULT_ADAPTER_NAME"]="NVIDIA"

from tools.visual_utils import open3d_vis_utils as Visualizer

dataset_cfg=EasyDict(yaml.safe_load(open('/home/rlab10/OpenPCDet/tools/cfgs/dataset_configs/zod_dataset_custom.yaml')))
class_names = ['Car', 'Pedestrian', 'Cyclist'] # use KITTI classes, bc we map with map_merged_classes()
file_path = '/home/rlab10/OpenPCDet/pcdet/datasets/zod/zod_dataset_custom.py' 
# /home/rlab10/OpenPCDet
ROOT_DIR = (Path(file_path).resolve().parent / '../../../').resolve()
data_path = ROOT_DIR / 'data' / 'zod'
ext = '.npy'
data = []
num_features = len(dataset_cfg.POINT_FEATURE_ENCODING.src_feature_list)

class DemoDatasetZod(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=True, root_path=None, logger=None, data_processor_flag=False, fov_mode=False, creating_pkl_infos=False):
        """
        Args:
            root_path:
            dataset_cfg:
            class_names:
            training:
            logger:
        """
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )

        self.split = self.dataset_cfg.DATA_SPLIT[self.mode]
        self.version = self.dataset_cfg.DATASET_VERSION 
        self.countries =  self.dataset_cfg.DATASET_COUNTRIES
        self.creating_pkl_infos = creating_pkl_infos
        self.zod_frames = ZodFrames(dataset_root=self.root_path, version=self.version)

        split_dir = self.root_path / 'ImageSets' / (self.split + '_' + self.version + '.txt') # ZOD
        self.sample_id_list = [x.strip() for x in open(split_dir).readlines()] if split_dir.exists() else None

        self.demo_zod_infos = []
        self.data_processor_flag = data_processor_flag
        self.fov_mode = fov_mode

        self.Tr_Zod_Lidar_to_Kitti_Lidar = np.array([[0, -1, 0],
                                                  [1,  0, 0], 
                                                  [0,  0, 1]])

        self.include_zod_data(mode='train')

    def include_zod_data(self, mode):
        if self.logger is not None:
            self.logger.info('DemoDatasetZod: Loading raw ZOD dataset')
        demo_zod_infos = []

        for info_path in self.dataset_cfg.INFO_PATH[mode]: # 'train', bc training=True
            info_path = self.root_path / info_path
            if not info_path.exists():
                continue
            # Read the data infos from zod_infos_train.pkl
            with open(info_path, 'rb') as f:
                infos = pickle.load(f)
                demo_zod_infos.extend(infos)

        self.demo_zod_infos.extend(demo_zod_infos)
        if self.logger is not None:
            self.logger.info('Total samples for ZOD dataset: %d' % (len(demo_zod_infos)))

        if not self.creating_pkl_infos:
            self. map_merged_classes()
        
    def map_merged_classes(self):
        if self.dataset_cfg.get('MAP_MERGED_CLASSES', None) is None:
            return
        
        #update class names in zod_infos
        map_merge_class = self.dataset_cfg.MAP_MERGED_CLASSES
        for info in self.demo_zod_infos:
            assert 'annos' in info
            info['annos']['name'] = np.vectorize(lambda name: map_merge_class[name], otypes=[str])(info['annos']['name'])

    def __len__(self):
        if self._merge_all_iters_to_one_epoch:
            if self.logger is not None:
                self.logger.info('zod_infos: %s', self.demo_zod_infos)
            return len(self.demo_zod_infos) * self.total_epochs 

        return len(self.demo_zod_infos)
    
    def get_lidar(self, idx, num_features=4):
        """
            Loads point cloud for a sample
                Args: 
                    index (int): Index of the point cloud file to get.
                Returns:
                    np.array(N, 4): point cloud
        """
        try:
            zod_frames_files = self.zod_frames[idx]
            lidar_core_frame = zod_frames_files.info.get_key_lidar_frame()
            pc = lidar_core_frame.read()

            if self.dataset_cfg.get('USE_VLS128_ONLY', False): 
                vls128_mask = pc.diode_idx < 128
                pc.points = pc.points[vls128_mask]
                pc.intensity = pc.intensity[vls128_mask]
                pc.diode_idx = pc.diode_idx[vls128_mask]
                pc.timestamps = pc.timestamps[vls128_mask]
        except Exception as e:
            print(f"Error loading Lidar for {idx}: {e}")

        if num_features == 4:
            # scale intensity to [0,1] from [0,255], bc at ZOD it isn't default
            pc.intensity = pc.intensity / 255
            # (x, y, z, intensity)
            points = np.concatenate((pc.points, pc.intensity.reshape(-1,1)), dtype=np.float32, axis=1)
        elif num_features == 5:
            pc.intensity = pc.intensity / 255
            points = np.concatenate((pc.points, pc.intensity.reshape(-1, 1), pc.diode_idx.reshape(-1, 1)), dtype=np.float32, axis=1)
        elif num_features == 3:
            points = pc.points
        else:
            raise NotImplementedError

        return points
    
    def get_calib(self, idx):
        zod_frame = self.zod_frames[idx]

        return zod_frame.calibration

    def get_fov_flag(self, pts_lidar, calib, camera=Camera.FRONT, lidar=Lidar.VELODYNE, use_kitti_fov=False):
        """
        Args:
            points (np.ndarray): LiDAR points in ZOD LiDAR coordinate system, shape (N, 3)
            calib: ZOD calibration object with camera and LiDAR extrinsics
            camera: ZOD Camera Enum (default: FRONT)
            lidar: ZOD Lidar Enum (default: VELODYNE)

        Returns:
            np.ndarray: Boolean mask (N,), True for point in FoV of camera
        """
        # Transformation LiDAR -> World
        t_lidar_to_world = calib.lidars[lidar].extrinsics
        # Transformation Camera -> World
        t_camera_to_world = calib.cameras[camera].extrinsics

        # Transformation World -> Camera
        t_world_to_camera= t_camera_to_world.inverse
        # Combine transformations LiDAR -> World -> Camera
        t_lidar_to_camera = Pose(t_world_to_camera.transform @ t_lidar_to_world.transform)

        #points_img = transform_points(pts_lidar[:, :3], t_lidar_to_camera.transform)
        points_img = transform_points(pts_lidar, t_lidar_to_camera.transform)

        # Only points with positive
        positive_depth = points_img[:, 2] > 0 # z>0
        mask = np.zeros(pts_lidar.shape[0], dtype=bool)
        if not np.any(positive_depth):
            return mask

        points_img_valid = points_img[positive_depth]

        if use_kitti_fov:
            kitti_fov = self.dataset_cfg.KITTI_FOV # (90°, 35°)
            _, fov_mask = get_points_in_camera_fov(fov=kitti_fov, camera_data=points_img_valid) # KITTI
        else:
            zod_fov = calib.cameras[camera].field_of_view # (120°, 67°)
            _, fov_mask = get_points_in_camera_fov(fov=zod_fov, camera_data=points_img_valid) # ZOD

        mask[positive_depth] = fov_mask

        return mask

    def prepare_demo_data(self, data_dict):
        print('DemoDatasetZod: prepare_demo_data() called')
        if 'gt_boxes' not in data_dict:
            assert 'gt_boxes' in data_dict, 'gt_boxes should be provided for demo visualization'
        else:         
            if data_dict.get('gt_boxes', None) is not None:
                selected = common_utils.keep_arrays_by_name(data_dict['gt_names'], self.class_names)
                data_dict['gt_boxes'] = data_dict['gt_boxes'][selected]
                data_dict['gt_names'] = data_dict['gt_names'][selected]
                gt_classes = np.array([self.class_names.index(n) + 1 for n in data_dict['gt_names']], dtype=np.int32)
                # already transformed 3D boxes LiDAR coord. + add number for the class
                gt_boxes = np.concatenate((data_dict['gt_boxes'], gt_classes.reshape(-1, 1).astype(np.float32)), axis=1)
                data_dict['gt_boxes'] = gt_boxes

            if data_dict.get('points', None) is not None:
                data_dict = self.point_feature_encoder.forward(data_dict)
            
            if self.data_processor_flag:
                print('DemoDatasetZod: data processing activated')
                data_dict = self.data_processor.forward(data_dict=data_dict)
    
        return data_dict

    def __getitem__(self, index):
        print('DemoDatasetZod: __getitem__ called')
        if self.merge_all_iters_to_one_epoch:
            index = index % len(self.demo_zod_infos)
        # works
        info = copy.deepcopy(self.demo_zod_infos[index])

        sample_idx = info['point_cloud']['lidar_idx']
        calib = self.get_calib(sample_idx)
        get_item_list = self.dataset_cfg.get('GET_ITEM_LIST', ['points'])

        input_dict = {
            'frame_id': sample_idx,
        }

        if 'annos' in info:
            annos = info['annos']
            annos = common_utils.drop_info_with_name(annos, name = 'DontCare')
            gt_names = annos['name']
            gt_boxes_lidar = annos['gt_boxes_lidar']

        input_dict.update({
            'gt_names': gt_names,
            'gt_boxes': gt_boxes_lidar
        })

        if 'points' in get_item_list:
            points = self.get_lidar(sample_idx, num_features=4)

            if self.fov_mode:
                print('DemoDatasetZod: fov mode activated')
                # only the points in the FOV of the camera are important for me
                fov_flag = self.get_fov_flag(points[:, 0:3], calib)
                points = points[fov_flag]
            
            points[:, :3] = points[:, :3] @ self.Tr_Zod_Lidar_to_Kitti_Lidar
            points[:, 2] -= self.dataset_cfg.LIDAR_Z_SHIFT

            input_dict['points'] = points

        data_dict = self.prepare_demo_data(data_dict=input_dict)

        return data_dict

def main(mode=str):
    logger = common_utils.create_logger()
    if mode == 'raw':
        logger.info('-----------------Quick Visualizer Demo of raw data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        demo_dataset = DemoDatasetZod(dataset_cfg=dataset_cfg, class_names=class_names, training=False, root_path=data_path, logger=logger, data_processor_flag=False, fov_mode=False, creating_pkl_infos=False)
        logger.info(f'Total number of samples: \t{len(demo_dataset)}')
      
        data_dict = demo_dataset[0] # 0 - 3711
        print('frame_id: ', data_dict['frame_id'])
        points = data_dict['points']
        gt_boxes = data_dict['gt_boxes']
        gt_names = data_dict['gt_names']

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes, gt_labels=gt_names, point_colors=None, draw_origin=True)
        logger.info('Demo visualization of raw data done.')

    elif mode == 'pre-processed':
        logger.info('-----------------Quick Visualizer Demo of pre-processed data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        demo_dataset = DemoDatasetZod(dataset_cfg=dataset_cfg, class_names=class_names, training=False, root_path=data_path, logger=logger, data_processor_flag=True, fov_mode=True, creating_pkl_infos=False)
        logger.info(f'Total number of samples: \t{len(demo_dataset)}')
       
        data_dict = demo_dataset[0] # 0 - 3711
        print('frame_id: ', data_dict['frame_id'])
        points = data_dict['points']
        gt_boxes = data_dict['gt_boxes']
        gt_names = data_dict['gt_names']

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes, gt_labels=gt_names, point_colors=None, draw_origin=True)
        logger.info('Demo visualization of pre-processed data done.')

    elif mode == 'val':

        logger.info('-----------------Quick Visualizer Demo of validation .pkl data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        pkl_path = ROOT_DIR / 'data' / 'zod' / 'zod_val_dataset.pkl'

        with open(pkl_path, 'rb') as f:
            demo_val_dataset = pickle.load(f)

        data_dict = demo_val_dataset[3582] # 0 - 3768
        points = data_dict['points']
        gt_boxes_lidar = data_dict['annos']['gt_boxes_lidar']
        gt_names = data_dict['annos']['name']
        gt_scores = data_dict['annos']['score']
        lidar_idx = data_dict['point_cloud']['lidar_idx']
        print(lidar_idx)

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes_lidar, gt_labels=gt_names, gt_score=gt_scores, point_colors=None, draw_origin=True, vc='val')
        logger.info('Demo visualization of pre-processed validation data done.')

    elif mode == 'train':

        logger.info('-----------------Quick Visualizer Demo of training .pkl data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        pkl_path = ROOT_DIR / 'data' / 'zod' / 'zod_train_dataset.pkl'

        with open(pkl_path, 'rb') as f:
            demo_train_dataset = pickle.load(f)

        data_list = demo_train_dataset[4] # specific sample from row no. in train.txt, e.g. (34-1) - 000067)
        # [0] - original; [1] - gt_sampling; [2] - flip_x; [3] - w_rotation; [4] - l_rotation; 
        # [5] - l_scaling; [6] - w_translation

        try:
            print("\nPlease select an option for the visualization:")
            print('[0] Original')
            print('[1] Ground Truth Sampling (gt_sampling)')
            print('[2] Flip along X-axis (flip_x)')
            print('[3] World rotation (w_rotation)')
            print('[4] Local rotation (l_rotation)')
            print('[5] Local scaling (l_scaling)')
            print('[6] World translation (w_translation)\n')
            user_choice = int(input('Your selection: '))
            if user_choice not in range(7):
                raise ValueError('Invalid selection')
        except ValueError as e:
            print(f"Error: {e}. Please enter a number between 0 and 6.")
            logger.error("Invalid user input for the visualization mode.")
            return
        
        switch_cases = {0: "original", 1: "gt_sampling", 2: "flip_x", 3: "w_rotation", 4: "l_rotation", 5: "l_scaling", 6: "w_translation"}
        
        selected_data = data_list[user_choice]
        points = selected_data['points']
        gt_boxes = selected_data['gt_boxes']
        gt_names = selected_data['gt_names']
        gt_scores = selected_data['cam_info']['score']
        lidar_idx = selected_data['frame_id']
        print(lidar_idx)

        if user_choice in [3, 4]: 
            noise_key = 'noise_world_rotation' if user_choice == 3 else 'noise_local_rotation'
            noise_rot = selected_data.get(noise_key, None)
            if noise_rot is not None:
                logger.info(f'Noise rotation for {switch_cases[user_choice]}: {noise_rot}')
        if user_choice == 5:
            noise_scale = selected_data.get('noise_local_scaling', None)
            if noise_scale is not None:
                logger.info(f'Noise scaling for {switch_cases[user_choice]}: {noise_scale}')
        if user_choice == 6:
            noise_trans = selected_data.get('noise_world_translation', None)
            if noise_trans is not None:
                logger.info(f'Noise translation for {switch_cases[user_choice]}: {noise_trans}')
        
        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes, gt_labels=gt_names, gt_score=gt_scores, point_colors=None, draw_origin=True, vc='')
        logger.info('Demo visualization of training data done.')
    
    if get_ipython():
        get_ipython().run_line_magic('reset', '-sf')
        #get_ipython().kernel.do_shutdown(restart=False)

if __name__ == '__main__':
    #main(mode='raw')
    #main(mode='pre-processed')
    main(mode='val')
    #main(mode='train')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2026-02-17 08:38:47,500   INFO  -----------------Quick Visualizer Demo of validation .pkl data-------------------------
2026-02-17 08:38:47,500   INFO  -----------------Quick Visualizer Demo of validation .pkl data-------------------------
2026-02-17 08:38:47,503   INFO  Mode for visualization: val
2026-02-17 08:38:47,503   INFO  Mode for visualization: val


000135


2026-02-17 08:38:58,877   INFO  Demo visualization of pre-processed validation data done.
2026-02-17 08:38:58,877   INFO  Demo visualization of pre-processed validation data done.
